# Práctica 1: Búsquedas en espacios de estados. Búsquedas no informadas.
* versión 1.0 (29 Ene 26)

<center><h3>
    Adrián Sequeiro Miramontes
</h3></center>

## Instrucciones

Esto es **Jupyter Notebook**, un documento que integra código Python en un archivo Markdown.
Esto nos permite ir ejecutando celdas de código poco a poco, así como generar automáticamente un informe bien formateado de la práctica.

Puedes añadir una celda con el botón **"Insert"** de la barra de herramentas, y cambiar su tipo con **"Cell > Cell Type"**

Para ejecutar una celda de código, la seleccionaremos y pulsaremos el botón **"▶ Run"** de la barra de herramentas.
Para pasar el documento a HTML, seleccionaremos **"File > Download as > HTML (.html)"**

Sigue este guión hasta el final. Ejecuta el código proporcionado paso a paso comprendiendo lo que estás haciendo y reflexionando sobre los resultados. Habrá preguntas intercaladas a lo largo del guión. Por favor, no modifiques ninguna linea de código excepto cuando se te pida explícitamente.

No olvides insertar tu **nombre y apellidos** en la celda superior.

Recuerda que esta práctica está planificada para **2 sesiones** y a lo largo de las mismas se discutirán grupalmente parte de las preguntas y problemas. 

## Entrega de la práctica

La fecha límite de entrega será la indicada en el Campus Virtual. La entrega consistirá de un único archivo comprimido con nombre `APELLIDOS_NOMBRE_Busquedas_noinformada.zip` que contenga los seguientes ficheros:

 * `APELLIDOS_NOMBRE_Busqueda_noinformada.html`: Archivo HTML fruto de la exportación del presente Notebook, con las preguntas respondidas al final del documento.
 * `APELLIDOS_NOMBRE_Busqueda_noinformada.ipynb`: Archivo fuente Jupyter Notebook.
 * Resto de archivos de códigos y datos que sean necesario para reejecutar tu práctica.

# Búsquedas en espacios de estados

El libro de consulta de referencia para esta práctica interactiva es el "Artificial Intelligence: A Modern Approach." (conocido como AIMA http://aima.cs.berkeley.edu/) de Russell y Norvig. En particular, el [capítulo 3](http://aima.cs.berkeley.edu/4th-ed/pdfs/newchap03.pdf) ofrece una panorámica completa sobre técnicas clásicas de resolución de problemas basadas en búsqueda en espacios de estados. En esta primera parte de la práctica interactiva, vamos a centrarnos en revisar dos estrategias de búsqueda no-informada o ciega utilizadas para resolver problemas de búsqueda en espacios de estados (búsqueda en anchura, búsqueda en profundidad), y trabajarás sobre el problema de 8-Puzzle introducido en la expositiva. 

## El problema del 8-Puzzle

![](https://ece.uwaterloo.ca/~dwharder/aads/Algorithms/N_puzzles/images/puz3.png)

El problema del 8-puzzle se plantea como un tablero 3x3 que contiene ocho fichas numeradas que se
pueden mover gracias a que una de las posiciones contiene un espacio en blanco o hueco. El objetivo
del juego es alcanzar el estado meta o objetivo indicado partiendo de un estado inicial dado, limitándose a
realizar movimientos permitidos del blanco. Una solución, entendida como un proceso de resolución, viene dada
por la secuencia de acciones o pasos que permiten alcanzar el estado meta.

Este problema forma parte de la familia de puzzles o rompecabezas de piezas deslizantes, que
pertenecen a la categoría de problemas NP. Tallas del problema más grandes, esto es, tableros con
mayor número de fichas puede hacer su resolución impracticable. Aun así, el 8-puzzle es útil para
probar algoritmos que implementen estrategias de búsqueda como los que utilizarás en esta práctica
pues se puede resolver en unos milisegundos a pesar de los 9!/2=181440 estados alcanzables.


Para familiarizarte con el problema, juega unas pocas partidas para resolver este **Juego 8-Puzzle online** disponible [aquí](https://appzaza.com/slide-puzzle/3x3) teniendo en cuenta que el estado meta es ![](./img/goal_default.png). Deberás pulsar el botón `New Game` para inciar cada nueva partida. 

¡A jugar! Aproximadamente 5-10 minutos... :-D


❓ **Pregunta 1**. Tras haber jugado (y así refrescado tus habilidades), aproximemos informalmente el problema mediante la resolución de los siguientes escenarios. Piensa cómo resolverías las configuraciones iniciales indicadas abajo y responde a las diferentes cuestiones. Si es necesario dibuja la secuencia de estados y operadores aplicados.

Configuraciones iniciales:

A) ![](./img/paso1.png)
B) ![](./img/pasos2b.png)
C) ![](./img/pasos2.png)

- 1.1) Indica la cadena de operadores realizada para alcanzar el estado meta. Representa en cada caso los diferentes estados que se van alcanzando en tu estrategia.
- 1.2) Explica qué conjunto de operadores has considerado y utilizado. ¿Todos tus operadores eran aplicables en cualquiera de los estados que has representado? Justifica tu respuesta.
- 1.3) ¿En algunas de las configuraciones tuviste que tener en cuenta varias alternativas? ¿Las tuviste que representar/dibujar? Si es el caso, documenta lo que hicieras en una celda Markdown.


Cabe destacar que las anteriores configuraciones eran muy sencillas y seguramente las habéis podido resolver casi mentalmente, con tan solo observar detenidamente la configuración inicial. En realidad, la cuestión a plantearse ahora es si has seguido una estrategia a ciegas o si habéis usado vuestro conocimiento del problema para dirigir mejor el conjunto de operadores y estados que realmente os acercaba a resolver el problema. Con total seguridad habéis aplicado vuestra *inteligencia natural*, que en ocasiones, no es sencilla de explicar. 

Sin embargo, en el caso de la *inteligencia artificial*, cuando queremos resolver problemas de búsqueda en espacios de estados como este, lo que se necesita son estrategias que sistemáticamente exploren el espacios de estados posibles y determinen una vez alcanzado un estado meta cuál es la secuencia de operadores que llevó a la solución. 

Dentro de ese conjunto de estrategias existe una categoría conocida como `búsquedas "no-informadas" o "a ciegas"`, entre las que se encuentran la *búsqueda en anchura/amplitud* y la *búsqueda en profundidad*. Estas estrategias se caracterizan por expandir el espacio de búsqueda, precisamente, de manera ciega, y explorarlo sistemáticamente siguiendo siempre el mismo criterio hasta encontrar un estado meta. 

Las siguiente dos figuras ilustran de manera abstracta el caso de un problema en el que se expanden los estados sucesores formando un árbol/grafo, que será el espacio de búsqueda de estados. Observa que ambas estrategias de búsqueda pueden desplegar exactamente el mismo árbol/grafo cuando los operadores sean los mismos. La diferencia por tanto radica en el orden de exploración de los estados. En el caso de la búsqueda en anchura, el recorrido se realiza por niveles, mientras que en la búsqueda en profundidad el recorrido se produce en vertical, profundizando, y sólo pasando a otra rama cuando ha completado la actual.


**Esquema de búsqueda en anchura/amplitud.**

![](./img/abstracta_anchura.png)

**Esquema de búsqueda en profundidad.**

![](./img/abstracta_profundidad.png)


Véamos cómo se construye nuestro espacio de búsqueda para un ejemplo concreto. 

❓ **Pregunta 2**. Dado el estado inicial indicado:
- Dibuja el espacio de estados que resultaría de expandir al menos 2 niveles completos. Aplica siempre los operadores en el mismo orden (p.ej., &larr;, &rarr;, &uarr;, &darr;) y representa en cada arco el operador que corresponda. Si fuera necesario añade nodos de manera justificada.
- Indica cuál podría ser el orden de visita de cada estado según se utilice una estrategia de búsqueda primero en anchura, o búsqueda en profundidad. Razona si cambiaría el orden de visita si se aplican los operadores en un orden diferente al que aplicaste arriba.

   ![](./img/tree2.png)
   

*Pega aquí los espacios de búsqueda que representaste con sus respectivas anotaciones.*


![](./estados/2-arbol_busqueda.png)

Tras el ejercicio manual anterior, habrás notado que puede ser laborioso o incluso poco práctico representar todo el espacio cuando las instancias del puzzle a resolver requiere soluciones de más de 1-2 movimientos. Para seguir examinando cómo funciona estas estrategias de búsqueda, ahora utiliza el **simulador RMIT-Búsquedas-8Puzzle** que se encuentra accesible online [aquí](https://tristanpenman.com/demos/n-puzzle/) para hacer trazas.

1) Edita los estados inicial y meta para que sean como en la siguiente figura:
   
   ![](./img/demo_primera_configuracion.png)
2) Selecciona el método de búsqueda primero en anchura y modo de control "Single-step".

3) Pulsa el botón "Start" para inicializar la búsqueda.

4) Observa la visualización del árbol, prestando atención al contenido de los nodos, y pulsa sucesivamente en el botón "Next" para estudiar su expansión.

5) Una vez finalice la búsqueda, realiza una captura de pantalla de la visualización resultante y ponla en una celda de markdown a continuación.

Repite el proceso pero en esta ocasión utilizando el método de búsqueda primero en profundidad.

Nota: Habrían otros simuladores similares disponibles (p.ej., https://deniz.co/8-puzzle-solver/). No obstante, ten en cuenta que las trazas pueden diferir debido a diferencias en la implementación del orden de los operadores que aplica el algoritmo de búsqueda.

*Pegar aquí la visualización resultante de la búsqueda en anchura* 
## BFS

![](./estados/2-arbol_busqueda-bfs.png)

*Pegar aquí la visualización resultante de la búsqueda en profundidad* 

## DFS (_usando LIFO)

![](./estados/2-arbol_busqueda-dfs.png)



❓ **Pregunta 3**. Utilizando el simulador RMIT, inspecciona las trazas para las estrategias de búsqueda primero en anchura y primero en profundidad para la configuraciones iniciales A), B), C).

- 3.1) ¿Coincide la traza que hiciste manualmente en la Pregunta 2 con la traza automática para la configuración **A**? Explica lo que observas y justifica su funcionamiento y posibles diferencias.

- 3.2) Consulta el material de teoría de las sesiones expositivas sobre completitud y optimalidad. ¿Qué significa que una estrategia de búsqueda sea completa? ¿Y qué significa que sea óptima?

- 3.3) Fíjate cuántos nodos se han expandido para resolver las configuración **A** y **C** en sendas estrategias utilizadas hasta el momento. ¿Dirías que las soluciones obtenidas son óptimas? Para esas instancias del problema, ambas estrategias han encontrado una solución, ¿qué dirías sobre su eficiencia, alguna sería más eficiente que la otra o serían igualmente eficientes? ¿podemos decir que ambas sean igualmente óptimas o alguna es más óptima que otra? ¿Alguna de las estrategías o ambas podría decirse que es/son estrategia/s de búsqueda óptimas de acuerdo a lo visto en la sesión expositiva? Justifica tus respuestas.

- 3.4) En base a las trazas realizadas y lo que conoces sobre las estrategías de búsqueda primero en anchura y primero en profundidad para resolver el problema del 8-Puzzle, razona para cada una ellas si sería Completa y Óptima. Justifica tus respuestas.

Documenta concisamente para justificar tus respuestas a las preguntas anteriores.


## Introducción a AIMA-Python y búsquedas ciegas


El libro de referencia AIMA viene con un conjunto de módulos Python que contiene muchos de los algoritmos implementados. Es por ello, que en la medida de lo posible utilizaremos sus implementaciones, que originalmente pueden consultarse en el repositorio de AIMA-Python https://github.com/aimacode/aima-python, y de los que hemos incorporado una copia adaptada a las necesidades de nuestra Asignatura en este boletín.

A partir de ahora, para trabajar con los algoritmos de búsqueda relevantes, primero nos familiarizaremos con las clases y funciones clave que nos permitirá resolver problemas mediante estrategias de búsqueda vistas en las sesiones expositivas. Será entonces que trabajaremos con el problema del 8-puzzle, que has estado ya inspeccionando mediante el uso de un simulador. Tras revisar cómo se implementan las estrategias de búsqueda ciega, procederás a estudiar el uso de heurísticas que permiten la implementación de búsqueda informada, pudiendo entonces estudiar el comportamiento las mismas. Finalmente, utilizarás la librería para implementar soluciones para los problemas del Viajante de Comercio.


En primer lugar, debes importar los módulos python especialmente preparados para esta Práctica 1.

In [1]:
# contiene algoritmos de búsqueda implementado en AIMA
from search_mod import *

# algunas funciones auxiliares
from helpers_mod import *

Puedes consultar el código de algunas clases y funciones clave mediante el uso de `psource(symbol)`. Por ejemplo, `psource(Problem)` mostrará el código de la clase Problem. 

In [2]:
psource(Problem)

Esta clase es fundamental pues, como mostraremos más abajo, se trata de una clase abstracta que puede extenderse para implementar la resolución de problemas concretos en base a los siguientes métodos:
- actions(state): devuelve una lista de operadores o acciones que son aplicables al estado actual
- result(state, action): devuelve el nuevo estado resultante de aplicar la acción action al estado state
- is_goal(state): devuelve cierto sii se ha alcanzando un estado meta u objetivo
- action_cost(s, action, s1): calcula el coste de aplicar la action para pasar del estado s al s1
- h(node): función heurística que se emplea en estrategias informadas. En una estrategia no informada o a ciegas debería retornar 0.

Cabe destacar que las estrategias de búsqueda se basan en búsquedas sobre grafos/árboles, y por lo tanto los algoritmos manejan nodos, que mantienen la relación entre los mismos, el coste del camino, y el estado que el nodo debe representar. De ahí, la clase `Node` tiene los atributos `state, parent, action, path_cost`.

Empieza por inspeccionar el código de las clases/funciones: `breadth_first_search, depth_first_recursive_search, depth_limited_search, expand, Problem, Node`.

In [3]:
psource(Node)
psource(Problem)
psource(breadth_first_search)
psource(depth_first_recursive_search)
psource(depth_limited_search)
psource(expand)

❓ **Pregunta 4**. Fíjate en la implementación de `breadth_first_search`, `depth_first_recursive_search`, y `depth_limited_search`. 

- 4.1) Indica las similitudes (p.ej., existe un bucle que explora un conjunto de nodos que llama frontera (nodos pendientes), expande el nodo para producir los sucesores (nodos-hijo)...) y también las diferencias. Explica cómo crees que dichas diferencias determinan el comportamiento de estas estrategias de búsqueda.

**Pista**: Fíjate en la estructura de datos para los nodos frontera, y consulta el pseudocódigo visto en las expositivas.


Como ya adelantábamos, la clase `Problem` puede extenderse para resolver problemas particulares.

La siguiente implementación permite usar los algoritmos de búsqueda que vienen implementados en AIMA-Python para resolver el problema del 8-puzzle (por el momento, ignora los métodos h que implementan heurísticas).

In [4]:
class EightPuzzle(Problem):
    """ The problem of sliding tiles numbered from 1 to 8 on a 3x3 board,
    where one of the squares is a blank, trying to reach a goal configuration.
    A board state is represented as a tuple of length 9, where the element at index i 
    represents the tile number at index i, or 0 if for the empty square, e.g. the goal:
        1 2 3
        4 5 6 ==> (1, 2, 3, 4, 5, 6, 7, 8, 0)
        7 8 _
    """

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        assert inversions(initial) % 2 == inversions(goal) % 2 # Parity check
        self.initial, self.goal = initial, goal
    
    def actions(self, state):
        """The indexes of the squares that the blank can move to."""
        moves = ((1, 3),    (0, 2, 4),    (1, 5),
                 (0, 4, 6), (1, 3, 5, 7), (2, 4, 8),
                 (3, 7),    (4, 6, 8),    (7, 5))
        blank = state.index(0)
        return moves[blank]
    
    def result(self, state, action):
        """Swap the blank with the square numbered `action`."""
        s = list(state)
        blank = state.index(0)
        s[action], s[blank] = s[blank], s[action]
        return tuple(s)
    
    
    def h(self, node): return 0
    
    

def inversions(board):
    "The number of times a piece is a smaller number than a following piece."
    return sum((a > b and a != 0 and b != 0) for (a, b) in combinations(board, 2))
    
    
def board8(board, fmt=(3 * '{} {} {}\n')):
    "A string representing an 8-puzzle board"
    return fmt.format(*board).replace('0', '_')



In [5]:
class Board(defaultdict):
    empty = '.'
    off = '#'
    def __init__(self, board=None, width=8, height=8, to_move=None, **kwds):
        if board is not None:
            self.update(board)
            self.width, self.height = (board.width, board.height) 
        else:
            self.width, self.height = (width, height)
        self.to_move = to_move

    def __missing__(self, key):
        x, y = key
        if x < 0 or x >= self.width or y < 0 or y >= self.height:
            return self.off
        else:
            return self.empty
        
    def __repr__(self):
        def row(y): return ' '.join(self[x, y] for x in range(self.width))
        return '\n'.join(row(y) for y in range(self.height))
            
    def __hash__(self): 
        return hash(tuple(sorted(self.items()))) + hash(self.to_move)

Puedes crear instancias de problemas del 8-Puzzle de la siguiente forma:

In [6]:
# Ejemplos de creación de instancias del problema del 8-puzzle
e1 = EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8))
e2 = EightPuzzle((1, 2, 3, 4, 5, 6, 7, 8, 0))
e3 = EightPuzzle((4, 0, 2, 5, 1, 3, 7, 8, 6))
e4 = EightPuzzle((7, 2, 4, 5, 0, 6, 8, 3, 1))
e5 = EightPuzzle((8, 6, 7, 2, 5, 4, 3, 0, 1))


y puedes proceder a resolver el problema mediante la estrategia de búsqueda deseada de la siguiente manera, que retorna el Node correspondiente al estado solución

In [7]:
# resolver una instancia particular con una estrategia de búsqueda concreta
# retorna la solución, como una instancia de Node
print(f'Resolver el problema {e1} con Búsqueda en Anchura')
sol = breadth_first_search(e1)
print ("Nodo solución: ", sol)

Resolver el problema EightPuzzle((1, 4, 2, 0, 7, 5, 3, 6, 8), (1, 2, 3, 4, 5, 6, 7, 8, 0)) con Búsqueda en Anchura
Nodo solución:  <(1, 2, 3, 4, 5, 6, 7, 8, 0)>


Para este problema, como el nodo de la solución es sencillamente el nodo meta, y éste era una entrada al problema, no resulta muy informativo. Sin embargo, sí podemos consultar los atributos de dicho nodo e incluso obtener los movimientos que condujeron al estado objetivo desde el estado inicial. 

Veamos cómo puedes obtener esta información.

In [8]:
# e imprimir informacion relevante utilizando los atributos y funciones siguientes
print (f'Coste del camino: {sol.path_cost:d}')
print (f'Estado solución: {sol.state:}')
print (f'Estado solución (tablero):')
print (board8(sol.state))
print (f'Lista de acciones: {path_actions(sol)}')
print (f'Numero de acciones en el camino: {len(path_actions(sol))}')
print (f'Lista de estados:')
print (path_states(sol))

# lista de estados formateado
for s in path_states(sol):
    print(board8(s))

Coste del camino: 23
Estado solución: (1, 2, 3, 4, 5, 6, 7, 8, 0)
Estado solución (tablero):
1 2 3
4 5 6
7 8 _

Lista de acciones: [4, 7, 6, 3, 4, 7, 8, 5, 4, 3, 6, 7, 4, 1, 2, 5, 8, 7, 6, 3, 4, 5, 8]
Numero de acciones en el camino: 23
Lista de estados:
[(1, 4, 2, 0, 7, 5, 3, 6, 8), (1, 4, 2, 7, 0, 5, 3, 6, 8), (1, 4, 2, 7, 6, 5, 3, 0, 8), (1, 4, 2, 7, 6, 5, 0, 3, 8), (1, 4, 2, 0, 6, 5, 7, 3, 8), (1, 4, 2, 6, 0, 5, 7, 3, 8), (1, 4, 2, 6, 3, 5, 7, 0, 8), (1, 4, 2, 6, 3, 5, 7, 8, 0), (1, 4, 2, 6, 3, 0, 7, 8, 5), (1, 4, 2, 6, 0, 3, 7, 8, 5), (1, 4, 2, 0, 6, 3, 7, 8, 5), (1, 4, 2, 7, 6, 3, 0, 8, 5), (1, 4, 2, 7, 6, 3, 8, 0, 5), (1, 4, 2, 7, 0, 3, 8, 6, 5), (1, 0, 2, 7, 4, 3, 8, 6, 5), (1, 2, 0, 7, 4, 3, 8, 6, 5), (1, 2, 3, 7, 4, 0, 8, 6, 5), (1, 2, 3, 7, 4, 5, 8, 6, 0), (1, 2, 3, 7, 4, 5, 8, 0, 6), (1, 2, 3, 7, 4, 5, 0, 8, 6), (1, 2, 3, 0, 4, 5, 7, 8, 6), (1, 2, 3, 4, 0, 5, 7, 8, 6), (1, 2, 3, 4, 5, 0, 7, 8, 6), (1, 2, 3, 4, 5, 6, 7, 8, 0)]
1 4 2
_ 7 5
3 6 8

1 4 2
7 _ 5
3 6 8

1 4 2
7 6 

Además de ver la traza, puedes también inspeccionar el código de `path_actions` y `path_states` para ver su implementación con el objetivo de entender cómo funciona internamente.

In [9]:
psource(path_actions)
psource(path_states)

❓ **Pregunta 5**. La clase EightPuzzle implementa el problema para el Puzzle, especificando la codificación del estado, los operadores y el modelo de transición entre estados. Inspecciona el código con detenimiento. Atendiendo a la implementación disponible, responde de manera razonada a las siguientes cuestiones:

- 5.1) Describe qué estructura de datos se utiliza y cómo se ha representado el estado en la implementación dada.
- 5.2) ¿Qué operadores (u acciones) pueden aplicarse en este problema? y ¿Cómo se implementa el modelo de transición (i.e.,  cómo se determinan los operadores aplicables y se lleva a cabo la acción adecuada)? Explícalo muy brevemente.
- 5.3) ¿Dónde se almacena el coste de la solución obtenida? ¿y dónde se almacena los operadores o acciones seguidas para alcanzar el estado meta?
- 5.4) Prueba a resolver otra instancia del problema del 8-puzzle no definida anteriormente. ¿Qué instancia has probado? ¿Cuál es el coste del camino de dicha resolución? Incluye el código de tus pruebas en la celda.


**Celda asociada a la Pregunta 5.4**
- 5.1) Utilízase un dicionario e os estados represéntanse mediante unha tupla de lonxitude 9
- 5.2) Os operadores aplicables son índices pertencentes aos da tupla (0-8)  e o modelo de transición modélase a partir da posición do espazo en branco. Dependendo da posición correspóndelle un conxunto de índices posibles polos que intercambiarse (codificados de maneira predefinida para cada posición no array de tuplas _moves_)
- 5.3) Gárdase no atributo path_cost do nodo da solución no algoritmo de búsqueda, que se calcula recursivamente. A acción de cada nodo na árbore de búsqueda gárdase no atributo action e, para obter os accións seguidas dende un estado, utilízase a función path_actions que calcula recursivamente as accións tomadas por cada nodo que integra a solución, a partir dos punteiros de cada un aos seus nodos pai.
- 5.4) A instancia que usei é esta (4, 6, 7, 3, 2, 1, 5, 8, 0) e o coste da solución é 24, é dicir, 24 movementos para chegar ao estado meta
  
  ```python
      e6 = EightPuzzle((4, 6, 7, 3, 2, 1, 5, 8, 0))
      sol = breadth_first_search(e6)
      print(f"Coste da solución: {sol.path_cost}")
      print(f"Lista de accións tomadas: {path_actions(sol)}")
      for i, state in enumerate(path_states(sol), 1):
        print(f"Estado {i}º: ")
        print(board8(state))
  ```

❓ **Pregunta 6**. Prueba a resolver las instancias del problema e1, e2 de nuevo, así como la instancia correspondiente a la configuración inicial A y C. Intenta resolver estas tres instancias con las estrategias `breadth_first_search`, `depth_first_recursive_search` y `depth_limited_search`. 

6.1) Crea una tabla resumen en [Markdown](https://denshub.com/es/howto-create-table-markdown/) sobre los resultados de las pruebas. Indica la lista de operadores y la longitud del camino solución. 

6.2) Explica qué pruebas fueron satisfactorias y cuáles no. Explica sobre las dificultades que has encontrado en la resolución de las pruebas, siempre razonando sobre el funcionamiento de los algoritmos.

 - 6.1)
   
|          e1                   |       Resultados     |  Operadores     |   Coste    |
| ----------------------------  | -------------------- | --------------- | ---------- |
|    breadth_first_search       |          Meta        |                 |     23     |
| depth_first_recursive_search  |     RecursionError   |                 |     Inf    |
|depth_limited_search(limit=100)|          Meta        |                 |     97     |

|          e2                   |       Resultados     |  Operadores     |   Coste    |
| ----------------------------  | -------------------- | --------------- | ---------- |
|    breadth_first_search       |         Meta         |      0          |     0      |
| depth_first_recursive_search  |         Meta         |      0          |     0      |
|depth_limited_search(limit=100)|         Meta         |      0          |     0      |

|          Configuración A      |       Resultados     |  Operadores     |   Coste    |
| ----------------------------  | -------------------- | --------------- | ---------- |
|    breadth_first_search       |         Meta         |     [8]         |     1      |
| depth_first_recursive_search  |         Meta         |      0          |     Inf    |
|depth_limited_search(limit=2)  |         Meta         |     [8]         |     1      |



In [10]:
a = EightPuzzle((1,2,3,4,5,0,7,8,6))
sol_1 = breadth_first_search(a)
#sol_2 = depth_first_recursive_search(e1)
sol_3 = depth_limited_search(a, limit= 2)
print(path_actions(sol_1), path_actions(sol_3))


[8] [8]


## Indicadores y comparación de estrategias


Ahora que eres capaz de ejecutar diferentes estrategias de búsqueda, podemos proceder a compararlas.

Para facilitar esta labor, disponemos de la función `report`, que nos permite obtener el resumen de estadísticas para un algoritmo o estrategia de búsqueda. Para colectar las estadísticas, esta función hace uso de la clase `CountCalls`, que envuelve un objeto `Problem` de tal forma que las llamadas a sus métodos se delegan al problema original a la vez que incrementa un contador.

Echa un vistazo al código de esta función y clase.

In [11]:
psource (report)
psource (CountCalls)
psource (report_counts)

Para obtener las estadísticas resultantes de la aplicación de múltiples estrategias a la resolución de un conjunto de instancias del problema, tan solo necesitamos establecer los parámetros de la función `report`.

Veamos un ejemplo en el que le indicamos que queremos resolver las instancias e1-e5 utilizando búsqueda en anchura, en profundidad (limitada).

In [12]:
def report(searchers, problems, verbose=True):
    """Show summary statistics for each searcher (and on each problem unless verbose is false)."""
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob   = CountCalls(p)
            soln   = searcher(prob)
            counts = prob._counts; 
            counts.update(actions=len(soln), cost=soln.path_cost)
            total_counts += counts
            if verbose: 
                report_counts(counts, str(p)[:40])
        report_counts(total_counts, 'TOTAL\n')

#Inclúo esta función modificada no código de proba

In [13]:
prob = CountCalls(e1)
print(breadth_first_search(prob).__counts)

AttributeError: 'Node' object has no attribute '__counts'

In [ ]:
report((breadth_first_search,  depth_limited_search), (e1, e2, e3, e4, e5))
#Aquí da que se path.inf en deep search o resultado é failure, pero

❓ **Pregunta 7**. Ejecuta la comparación de las anteriores estrategias de búsqueda y responde a las siguientes cuestiones de manera justificada en relación a las medidas obtenidas.

- 7.1) Explica brevemente qué significan los indicadores *nodes y cost* que muestra la salida de `report`.

- 7.2) Discute de manera justificada el rendimiento de los algoritmos en términos del coste del camino/solución y el número de nodos explorados.

- 7.3) Crea un conjunto de instancias que puedan resolverse mediante la estrategia búsqueda primero en profundidad, y realiza la comparativa. Discute los resultados.


In [ ]:
# Aquí tu código de pruebas



- 7.1) _Nodes_ son todos os nodos explorados durante a búsqueda (aínda que non teñen por que ser todos expandidos/visitados) e _cost_ e o coste acumulado de todos as accións ata chegar á solución, no caso do _EightPuzzle_, constitúen todos os movementos/intercambios de índices no 3x3.

- 7.2) O algoritmo de búsqueda en anchura vemos que sempre chega á solución, que sempre é óptima, pero ca contrapartida de explorar un gran número de nodos. Por exemplo, para unha solución de 20 nodos de lonxitude (xa que o custo é unitario, 1 por movemento dos índices da tupla do puzzle), explorou 121.000 nodos e para unha solución de 30 483.000.

Pola contra, a búsqueda en profundidade non recursiva e limitada a unha certa altura (10 nodos, neste caso) explora moitos menos nodos (na orde dos miles), pero é incapaz de atopar a unha solución se esta é maior que o seu límite de profunidade (neste caso, 10 nodos). Tampouco garantiza optimalidade, é dicir, escolle a primeira solución que atopa.

- 7.3)


# El problema de Viajante de Comercio (VC)


El objetivo de este bloque es modelar e implementar un agente inteligente que sea capaz de resolver
el problema del VC mediante estrategias de búsqueda A*, haciendo uso de AIMA como te has familiarizado anteriormente.


### Definición del problema

El problema del viajante de comercio (VC) es el problema de la persona que quiere vender un producto, y que para ello quiere encontrar el viaje más corto posible a través de las ciudades de los clientes, haciendo una única visita a cada una, empezando y acabando el recorrido en su propia ciudad (recorrido circular desde la ciudad inicial).

Típicamente, el problema parte de una representación mediante un grafo ponderado G=(N, A), donde N es el conjunto de n=|N| nodos (ciudades), y siendo A el conjunto de arcos conectando los nodos. Cada arco (i, j) ∈ A tiene asignado un peso d_ij que representa la distancia entre las ciudades i y j.

Para facilitar vuestra labor de implementación, os proporcionamos la clase `Localizaciones`, que permite cargar las localizaciones GPS que representan los vértices del grafo G de N ciudades, y permite calcular de manera transparente la distancia entre cualquier par de ciudades.


Inspecciona el código de carga de localizaciones mediante `psource(Localizaciones)`

In [ ]:
psource (Localizaciones)

Fíjate que por defecto se carga el fichero `./data/grafo8cidades.txt`, que contiene las coordenadas GPS de 8 ciudades gallegas, siendo Santiago de Compostela la primera de ellas. La primera línea de estos ficheros indica el número de ciudades n, mientras que cada una de las líneas sucesivas especifican las coordenadas de cada ciudad, especificadas como coordenadas GPS (latitud y longitud en grados).

Fíjate que las distancias en Google Maps van a ser más largas que las nuestras pues nosotros solo tenemos en cuenta la curvatura de la tierra en línea recta mientras que el servicio de mapas toma en cuenta otros aspectos relevantes sobre la existencia de carretera o vía así como otras características geográficas. 

![8cidades.png](./img/8cidades.png)

Puedes cargar otro fichero haciendo uso del parámetro `filename` como se muestra a continuación.

In [ ]:
g1=Localizaciones(filename='./data/grafo8cidades.txt')
print (g1.distancia(0,1))
g2=Localizaciones(filename='./data/grafos10_10/grafo_1.txt')
print (g2.distancia(0,1))

### Implementación del problema VC


El VC se reduce al problema de crear el circuito Hamiltoniano de longitud mínima sobre el grafo G. La solución a una instancia del problema del VC puede representarse como una permutación de los índices de las ciudades, donde lo importante es el orden de visita, que determinará el coste del viaje en términos de la distancia recorrida total. 

De este modo, el problema pertenece a la categoría de problemas NP, pues puede haber n permutaciones que se corresponden al espacio de búsqueda posible. Esto hace que resolver instancias de problemas con muchas ciudades (n grande) haga el problema impracticable y éste pueda beneficiarse de búsquedas informadas, que guíen inteligentemente el proceso para reducir el espacio de búsqueda y por tanto el esfuerzo computacional.

Véamos cómo podríamos plantear nuestra implementación del VC. Para hacer la explicación más manejable, supongamos que tomamos las 4 primeras ciudades de nuestro problema de las 8 ciudades gallegas.

 ![4cidades.png](./img/4cidades.png)


En primer lugar necesitamos determinar una representación adecuada de nuestros estados en el espacio de búsqueda. Aunque se puede hacer de varias formas, una forma razonable sería disponer de una tupla cuyos elementos serían los índices de la ciudades según el orden en que han sido visitadas. 

Por ejemplo, la tupla (0, 1, 3, 2 , 0) representaría un estado que indicaría el siguiente camino: Santiago de Compostela- A Coruña - Lugo - Villalba - Santiago de Compostela. Fíjate que dicho camino es realmente un ciclo, pues la primera y última ciudad son la 0, Santiago de Compostela, y por tanto, podría ser un estado meta. Viendo el mapa de ejemplo, sabemos que pueden haber varios estados meta, y el que sería considerado como solución óptima correspondería a la secuencia (0, 1, 2, 3, 0).

Cabe hacerse ahora algunas preguntas clave. ¿Inicialmente dónde estamos y cuántas ciudades hemos visitado? Efectivamente, de partida estaremos en la ciudad 0, y no habremos visitado ninguna otra ciudad todavía, así que podríamos representar nuestro estado inicial como (0,). 

¿Y si entonces visitamos una segunda ciudad? Pues podríamos ir a A Coruña (1), Villalba (2) o a Lugo (3), lo que daría lugar a los siguientes estados sucesores (0, 1), (0, 2) o (0, 3) respectivamente. 

Imagina que nuestra estrategia selecciona A Coruña. En ese caso, el estado resultante sería (0, 1). A partir de este, nota que los sucesores serían (0, 1, 2) y/o (0, 1, 3), fruto de considerar las otras dos ciudades que no han sido todavía visitadas. Debes fijarte que en estos casos, los estados representan la secuencia de índices mediante una tupla de forma que se va construyendo paso a paso. Y cabe destacar que sabemos que no son estados meta porque no han visitado todas las ciudades y cerrado el ciclo todavía.

A continuación mostramos un árbol de búsqueda representando nuestro espacio de búsqueda. Reflexiona lo que representan los nodos y los arcos. En el caso del VC, ¿dónde representarías el coste del camino? Piénsalo, e indicalo en la pregunta 5 cuando explicas tu implementación.

![](./img/arbolbusqueda.png)

❓ **Pregunta 8**. Implementa el problema del Viajante de Comercio siguiendo las indicaciones y responde a las cuestiones correspondientes.

Tu implementación `ProblemaViajanteComercio` debe extender la clase `Problem` y sobreescribir los métodos `actions, result, is_goal, action_cost` según la representación del estado discutida arriba. 
Por el momento, considera el grafo de las 8 ciudades gallegas y una implementación nula de la heurística, h=0, puesto que por el momento sólamente vas a utilizar las estrategias de búsqueda no informada o a ciegas. 

- 8.1) Explica cómo has implementado los operadores así como el modelo de transición que gobierna la expansión o generación de sucesores de los estados en el espacio de búsqueda. Explica brevemente cuáles han sido los aspectos clave en tu implementación de `actions, result, is_goal, action_cost`.
- 8.2) Verifica la correctitud de tu implementación resolviendo varias instancias del problema del viajante mediante las estrategias no informadas vistas en este tutorial. Haz una tabla donde muestres los nodos y el coste del camino obtenido para problema.


In [ ]:
psource(Problem)

In [ ]:
# Celda asociada a Pregunta 8
# Completa la implementación

class ProblemaViajanteComercio(Problem):
    """ Problema Viajante de Comercio
    """    
    #localizaciones é unha instancia da clase Localizaciones que nos permite calcular as distancias entre cidades e numero
    # e número e o número de cidades
    def __init__(self, initial, localizaciones):
        self.initial = (initial,)
        self.cidades = [i for i in range(localizaciones.nciudades)]
        self.localizaciones = localizaciones
        
    
    def actions(self, state: tuple()):
        """devuelve la lista de ciudades no visitadas como acciones potenciales."""
        if len(state) < len(self.cidades):
            return tuple(cidade for cidade in self.cidades if cidade not in state)
        elif len(state) == len(self.cidades):
            return (0,)
        else:
            return []
    
    def result(self, state: tuple(), action: int):
        """produce el nuevo estado al anadir la accion seleccionada"""
        if action not in self.actions(state):
            import warnings
            warnings.warn(f"Accion inválida {action}. Accións válidas: {self.actions(state)}")
            return state
            
        state = state + (action,)
        return state
    
    def is_goal(self, state: tuple()): 
        if len(state)< len(self.cidades)+1:
            return False
        return state[0] == state[-1] == self.initial[0]


    # s e s1 son os estados
    def action_cost(self, s: tuple(), action, s1: tuple()= None):
        """The distance (cost) to go from s to s1."""       
        *_, ultimo = s
        return self.localizaciones.distancia(ultimo, action)
    
    def h(self, node): 
        return 0
    

In [ ]:
# Celda de código
localizaciones = Localizaciones(filename="./data/grafo8cidades.txt")
TSP = ProblemaViajanteComercio(0, localizaciones)
from random import randint

#Probas unitarias
for i in range(100):
    rango = randint(2,9)
    valores = [randint(0,7) for _ in range(rango)]
    resultado = randint (0,7)
    coste_accion = randint (0,7)
    print(valores, rango, f"O resultado de aplicar a acción {resultado} é {TSP.result(tuple(valores), resultado)}", 
        f"E o coste de ir a  {coste_accion} é {TSP.action_cost(tuple(valores), coste_accion)}")
    if valores[0] == valores[-1]:
        print(TSP.is_goal(tuple(valores)))
    #print(TSP.actions(tupla)

#8.2 Práctica problema_viajante con algoritmos de búsqueda
print(breadth_first_search(TSP))
print(depth_first_recursive_search(TSP))
print(depth_limited_search(TSP))

In [ ]:
TSP.result((1,2,3,4), 4)

In [ ]:
psource(Node)
psource(breadth_first_search)
psource(depth_first_recursive_search)
psource(depth_limited_search)
psource(expand)

**************************************************************

# Respuestas a los ejercicios y evaluación

**RECUERDA:** No te olvides de escribir tu nombre y apellidos en la segunda celda del presente documento.

Incluye las celdas que consideres oportunas para que sea legible. Incluye la formulación de las preguntas para una mejor comprensión y facilitar su corrección.

La práctica se evaluará mediante un cuestionario de evaluación, sin poder utilizar ningún tipo de asistencia. El cuestionario de evaluación se realiza habitualmente en la primera sesión de la próxima práctica, y se centrará en la resolución por tu parte de diversas cuestiones prácticas relacionadas tanto con los aspectos conceptuales como de implementación de tu práctica, pudiendo ser necesaria la ejecución, adaptación y modificación de la misma durante la duración de la prueba. 

Independientemente del cuestionario de evaluación, las preguntas planteadas deben responderse en el notebook de manera clara y precisa. Esas preguntas generales están diseñadas para formarte, y te servirán para razonar y reflexionar sobre el tema, así como también para fomentar una discusión constructiva con los docentes durante las sesiones. No se dará por superada la práctica si las respuestas en el notebook no son adecuadas. 

Recuerda que el plagio/copia completa o parcial de prácticas de otros compañeros acarrea no solo el suspenso de la práctica en curso sino también de la asignatura en dos convocatorias.

**************************************************************

## **Exercicio 1**

<hr>

1.1)
- **Imaxe A**: Neste caso utilice o operador &darr; para subir o 6 e alcanzaríamos o estado meta
- **Imaxe B**:  Neste caso utilizo o operador &rarr; e alcanzo o estado e  despois utilizamos o operador &darr; e alcanzamos o estado meta, pasando por estes estados:

<span style="display:inline-flex; align-items:center;">
  <img src="./img/pasos2b.png" alt="">
  <span style="margin: 0 0.5em;">→</span>
  <img src="./estados/1.1-B.png" alt="">
  <span style="margin: 0 0.5em;">→</span>
  <img src="./estados/meta.png" alt="">
</span>

- **Imaxe C**: Neste caso, utilizamos o operador &darr; dúas veces e alcanzamos o estado meta, pasando por estes estados:

<span style="display:inline-flex; align-items:center;">
  <img src="./img/pasos2.png" alt="">
  <span style="margin: 0 0.5em;">→</span>
  <img src="./estados/1.1-C.png" alt="">
  <span style="margin: 0 0.5em;">→</span>
  <img src="./estados/meta.png" alt="">
</span>


1.2) Utilice os operadores &darr; e &rarr;. E non, non en tódolos casos, pódese usar tódolos operadores, depende de en que direccións se poida mover o bloque. Por exemplo, nas esquinas só dous operadores son utilizables e no centro os 4, mentres que non bloques laterias dos lados do cadrado podemos usar ata 3.

1.3) Na primeira imaxe, non había ningunha alternativa mellor e máis rápida que baixar o 6. No segundo e terceiro, realmente tampouco había, porque ao ser dous movementos, todas as alternativas eran máis costosas en tempo e movementos


                                                                      
                                                                     

# <u>**Exercicio 2** </u>

<hr>

## BFS (cola con estratexia FIFO ou LIFO)
Se se utilizase esta estratexia, se percorren os nodos de todos os nodos do nivel fronteira, antes de expandir o primeiro o último deles (segundo estratexia _FIFO_ ou _LIFO_ respectivamente).

Polo tanto a orde de visita sería, #1, #2, #3, #4

### DFS 
Neste caso, percorremos recursivamente ou non recursivamente os fillos dos nodos visitados antes que os seus irmáns. 

Entón a orde de visita sería #2, os fillos de #2, #3, os fillos de #3 e, por último, #4, que é a solución.

 Como se ve, DFS pode chegar a ser bastante ineficiente para atopar solucións cercanas ao estado inicial

## **Exercicio 3**

<hr>

- 3.1) Si, respecto ao de bfs, xa que eu manualmente escribín todas as posibles combinacións (_obviando os fillos da solución e as combinacións non factibles_) e como a solución é o último dos nodos fillos de profundidade 1, no caso de búsqueda en anchura, expandirá o nodo 1 e 2 de profundidade 1, antes de chegar ao 3.

&emsp; No caso de búsqueda en profundidade, como comeza á expandir polo último nodo engadido á lista, neste caso o nodo solución, non expande os fillos de 1 e 2.

- 3.2) Que sexa _completa_ significa que asegura unha solución, se existe, e que sexa _óptima_ significa que atopa a mellor solución

- 3.3) En ambos casos, as estratexias son completas, neste caso (_porque atopan unha solución_) e son óptimas, xa que búsqueda en anchura, ao percorrer cada nivel enteiro, garantiza atopar a solución máis óptima, mentres que en profundidade só devolve a **primeira** solución atopada. Por iso se dí que en anchura é unha estratexia de búsqueda óptima.

Neste caso, non obstante, a estratexia en profundidade é máis eficiente, xa que percorre e expande menos nodos, xa que, casualmente, a solución é aplica &darr; dúas veces, que, segundo á orde (&larr;, &rarr; &uarr;, &darr;) é o último nodo expandido en cada nivel, que xustamente é o primeiro que percorre o DFS

- 3.4) A estratexia en anchura, polo xa dito, é óptima, e polo tamén completa, mentres que a estratexia en profundidade non garantiza optimilidade nin completitude, porque pode quedar atascado en bucles infinitos, se existe un espazo de combinacións infinitas ou repetidas.

Por exemplo, no noso exemplo, o dfs, se non marcamos un límite de profundidade, expandiría a rama do primeiro fillo, indefinidamente (debido á combinacións repetidas) e nunca chegaría á solución na rama do terceiro fillo

<h2><b>Exercicio 4</b></h2>

<hr>

No _depth_limited_search_ usan unha pila auxiliar (ou cola LIFO) para sacar o último fillo e expandilo, antes que os seguintes fillos, mentres que na _breadth_first_search_ utilizan unha cola para extraer primeiro todos os fillos, comprobar se os seus estados son solución, marcalos como visitados e engadilos á cola, de maneira que se van expandir por orde de entrada. No _depth_first_recursive_search chámase a función recursivamente con cada novo nodo. 

Nos tres algoritmos utilízase un bucle que vai percorrendo todos os nodos da fronteira, pero a lóxica é distinta: nos algoritmos de búsqueda en profundidade, popéase o primeiro nodo da lista e comprobáse se é solución (todo, por agora, igual que en anchura) e engádense os fillos á lista, no caso da cola engádense pola esquerda (xa que se removen pola dereita) e no caso da pila engádense á dereita para manter esa orde LIFO. De esta maneira, a busca en profundidade expande o último fillo de cada un dos nodos (puidendo chegar á unha solución ou quedarse atascado nun ciclo, de ahí a función _is_cycle()_) e se non chega a unha solución, expande o penúltimo do penúltimo nivel, logo do antepenúltimo...

Á diferenza entre o _depth_first_recursive_search e o depth_limited_search é que este último comproba á profundidade (mediante o argumento '_limit_'), e se o nodo excédea, poda. No caso de _depth_first_recursive_search_, ao ser unha función recursiva, ten 3/2 casos base: se o nodo é solución, se é nulo (só para o inicial) ou se chegamos á un ciclo, que nese caso devolve erro